![An interactive LADAL notebook.](https://slcladal.github.io/images/uq1.jpg)


***

Please copy this Jupyter notebook so that you are able to edit it.

Simply go to: File > Save a copy in Drive.

If you want to run this notebook on your own computer, you need to do 2 things:

1. Make sure that you have R installed.

2. You need to download the [bibliography file](https://slcladal.github.io/bibliography.bib) and store it in the same folder where you store the Rmd file.

Once you have done that, you are good to go.

***

# Converting PDFs to txt files with R

This tutorial shows how to extract text from one or more pdf-files and then saving the text(s) in txt-files on your computer. The RNotebook for this tutorial can be downloaded [here](https://slcladal.github.io/pdf2txt.Rmd). If you want to render the Rmarkdown notebook on your machine, i.e. knitting the document to a html or pdf file, you need to make sure that you have R installed and you also need to download the [bibliography file](https://slcladal.github.io/bibliography.bib) and store it in the same folder where you store the Rmd file.  

The aim of this tutorial is to showcase how to extract text from pdf documents that are stored on your computer. The aim is not to provide a ready-made tool or to provide a perfect text extraction but rather to show and exemplify an easy and common method for extracting text that can be applied to a large number of pdf-documents.

**Preparation and session set up**

This tutorial is based on R. If you have not installed R or are new to it, you will find an introduction to and more information how to use R [here](https://slcladal.github.io/intror.html). For this tutorials, we need to install certain *packages* from an R *library* so that the scripts shown below are executed without errors. Before turning to the code below, please install the packages by running the code below this paragraph. If you have already installed the packages mentioned below, then you can skip ahead and ignore this section. To install the necessary packages, simply run the following code - it may take some time (between 1 and 5 minutes to install all of the libraries so you do not need to worry if it takes some time).


In [ ]:
# set options
options(stringsAsFactors = F)          # no automatic data transformation
options("scipen" = 100, "digits" = 12) # suppress math annotation
# install packages
install.packages("tidyverse")
install.packages("here")


Unfortunately the package `pdftools` has to be installed separately (as shown below).



In [ ]:
system2('sudo', 'apt-get install libgsl0-dev')
install.packages("pdftools", repos='http://cran.us.r-project.org')


Next we activate the packages.



In [ ]:
# activate packages
library(pdftools)
library(tidyverse)
library(here)


Once you have installed RStudio and have also initiated the session by executing the code shown above, you are good to go.

**How to use the RNotebook for this tutorial**

To follow this tutorial interactively (by using the RNotebook - or Rmd for short), follow the instructions listed below.

**Data and folder set up**

1. Create a folder somewhere on your computer
2. In that folder create a sub-folder called *data*
3. In that *data* folder, create a subfolder called *PDFs* 
4. Download and  save the following pdf-files in that *PDFs* folder: 
[pdf0](https://slcladal.github.io/data/PDFs/pdf0.pdf), [pdf1](https://slcladal.github.io/data/PDFs/pdf1.pdf), [pdf2](https://slcladal.github.io/data/PDFs/pdf2.pdf), and [pdf3](https://slcladal.github.io/data/PDFs/pdf3.pdf).

**R and RStudio set up**

1. Download the [RNotebook](https://slcladal.github.io//pdf2txt.Rmd) and save it in the folder you have just created
2. Open RStudio
3. Click on `File` in the upper left corner of the R Studio interface
4. Click on `New Project...`
5. Select `Existing Directory`
6. Browse to the folder you have just created and click on `Open`
7. Now click on `Files` above the lower right panel
8. Click on the file `pdf2txt.Rmd`
   + The Markdown file of this tutorial should now be open in the upper left panel of RStudio. To execute the code which prepare the session, load the data, create the graphs, and perform the statistics, simply click on the green arrows in the top right corner of the code boxes.
   + To render a PDF of this tutorial, simply click on `Knit` above the upper left panel in RStudio.


**Upload your data to Google Colab**

The code chunk below allows you to upload two files from your own computer. To be able to load your own data, you need to click on the folder symbol to the left of the screen:

![Colab Folder Symbol](https://slcladal.github.io/images/ColabFolder.png)

Then on the upload symbol. 

![Colab Upload Symbol](https://slcladal.github.io/images/ColabUpload.png)

Next, upload the files you want to analyze and then the respective files names in the `file` argument of the `scan` function. When you then execute the code (like to code chunk below, you will upload your own data.


In [ ]:
pdf0 <- "pdf0.pdf"
pdf1 <- "pdf1.pdf"
pdf2 <- "pdf2.pdf"
pdf3 <- "pdf3.pdf"
# inspect
pdf0


To apply the code and functions below to your own data, you will need to modify the code chunks and replace the data we use here with your own data object. 

Now that we have loaded some data, we can go ahead and extract the text from the pdf-documents.


# Extract text from one pdf

The pdf we will convert is a [pdf of the Wikipedia article about corpus linguistics](https://slcladal.github.io/data/PDFs/pdf0.pdf). The first part of that pdf is shown below.

![An example pdf document](https://slcladal.github.io/images/pdf0.png)


Given that the pdf contains tables, urls, reference, etc., the text that we will extract from the pdf will be rather messy - cleaning the content of the text would be another matter (it would be data processing rather than extraction) and we will thus only focus on the conversion process here and not focus on the data cleaning and processing aspect. 

We begin the extraction by defining a path to the pdf. Once we have defined a path, i.e. where R is supposed to look for that file, we continue by extracting the text from the pdf.


In [ ]:
# you can use an url or a path that leads to a pdf document
pdf_path <- "pdf0.pdf"
# extract text
txt_output <- pdftools::pdf_text(pdf_path) %>%
  paste0(collapse = " ") %>%
  paste0(collapse = " ") %>%
  stringr::str_squish() 


In [ ]:
# show data: First 1000 characters of the extracted text from a pdf of the wikipedia article on corpus linguistics.
txt_output %>%
  substr(start=1, stop=1000) %>%
  as.data.frame()


# Extracting text from many pdfs

To convert many pdf-files, we write a function that preforms the conversion for many documents.


In [ ]:
convertpdf2txt <- function(dirpath){
  files <- list.files(dirpath, full.names = T)
  x <- sapply(files, function(x){
  x <- pdftools::pdf_text(x) %>%
  paste0(collapse = " ") %>%
  stringr::str_squish()
  return(x)
    })
}


We can now apply the function to the folder in which we have stored the pdf-files we want to convert. In the present case, I have stored 4 pdf-files of Wikipedia articles in a folder called *PDFs* which is located in my *data* folder as described in the sectionabove which detailed how to set up the Rproject folder on your computer). The output is a vector with the texts of the pdf-files. 



In [ ]:
# apply function
txts <- convertpdf2txt(here::here("data", "PDFs/"))


In [ ]:
# show data: First 1000 characters of the extracted texts from pdfs of selected wikipedia articles.
txts %>%
  substr(start=1, stop=1000) %>%
  as.data.frame()


The table above shows the first 1000 characters of the texts extracted from 4 pdf-files of Wikipedia articles associated with language technology (*corpus linguistics*, *linguistics*, *natural language processing*, and *computational linguistics*). 

# Saving the texts

To save the texts in txt-files on your disc, you can simply replace the predefined location (the data folder of your Rproject located by the string `here::here("data")` with the folder where you want to store the txt-files and then execute the code below. Also, we will name the texts (or the txt-files if you like) as *pdftext* plus their index number.


In [ ]:
# add names to txt files
names(txts) <- paste0(here::here("data","pdftext"), 1:length(txts), sep = "")
# save result to disc
lapply(seq_along(txts), function(i)writeLines(text = unlist(txts[i]),
    con = paste(names(txts)[i],".txt", sep = "")))


If you check the data folder in your Rproject folder, you should find 4 files called *pdftext1*, *pdftext2*, *pdftext3*, *pdftext4*.

# Citation & Session Info 

Schweinberger, Martin. 2021. *Converting PDFs to txt files with R*. Brisbane: The University of Queensland. url: https://slcladal.github.io/pdf2txt.html.


In [ ]:
sessionInfo()

